In [1]:
# codigo concentrado del proyecto

In [2]:
# Importamos librerias
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set()

In [47]:
def estandarizacion_informacion(data):
    # eliminamos los NA que determinamos
    df = data
    df = df.dropna(subset=['Electrical'])
    # rellenamos los NA con 0
    df = df.fillna(0)

    
    # Funcion para obtener si la casa ah sido remodelada o no
    df['Remodelacion'] = df.YearRemodAdd == df.YearBuilt
    df['Remodelacion'] = df['Remodelacion'].astype('str')
    df['Remodelacion'] = df['Remodelacion'].replace({'True':0,'False':1})

    # Funcion para la obtencion de los años de construccion de la casa y garage
    df['AñosConstCasa'] = 2021 - df['YearBuilt'] 
    df['AñosConstGar']= 2021 - df['GarageYrBlt']
    df['AñosConstGar'] = df['AñosConstGar'].replace({2021:0})

        
    # Obtencion de dummies
    dummies = ['MSZoning','Street','Alley','LandContour','LotConfig','Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','Foundation','Heating','CentralAir','Electrical','GarageType','PavedDrive','MiscFeature','SaleType','SaleCondition']
    df = pd.get_dummies(data=df,columns=dummies)

    # Tratamiento de datos de calificacion a numero
    # Revisando las descripciones en el diccionario, vemos que para todas estas variables califican alguna de las caracteristicas de la casa
    # con lo que nos permite asignarle una calificacion de menor a mayor, dependiendo de cada una de las variables
    columnas_valores_reempl = ['BsmtFinType2','HeatingQC','KitchenQual','Functional','FireplaceQu','GarageFinish','GarageQual','GarageCond','PoolQC','Fence','LotShape','Utilities','LandSlope','ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1']
    diccionario = {'Reg':3,'IR1':2,'IR2':1,'IR3':0,'Typ':8,'Min1':7,'Min2':6,'Mod':5,'Maj1':4,'Maj2':3,'Sev':2,'Sal':1,'AllPub':4,'NoSewr':3,'NoSeWa':2,'ELO':1,'Gtl':2,'Mod':1,'Sev':0,'Fin':3,'RFn':2,'Unf':1,'NA':0,'Gd':4,'Av':3,'Mn':2,'No':1,'GLQ':6,'ALQ':5,'BLQ':4,'Rec':3,'LwQ':2,'Unf':1,'Ex':4,'Gd':3,'TA':2,'Fa':1,'Po':0,'GdPrv':4,'MnPrv':3,'GdWo':2,'MnWw':1}

    for columna in columnas_valores_reempl:
        df[str(columna)] = df[str(columna)].replace(diccionario)

    # Asegurarnos de que los datos estén en formato numero
    for i in df.columns:
        df[i].astype('int64')
    
    # Reduccion del numero de columnas
    # Baño:
    # para reducir un poco el numero de columnas, haremos algunas agrupaciones
    # Sumaremos el numero total de baños completos
    df['TotBaños'] = df['BsmtFullBath']+df['FullBath']
    # Sumaremos el numero total de medios baños
    df['TotMedBaño']=df['BsmtHalfBath']+df['HalfBath']
    # El sotano tiene varias medidas, por lo que trataremos de crear una sola variable que las pueda explicar todas
    # vamos a multiplicar la calificion del sotano por la de tamaño correspondiente
    df['Bsmt1Cal'] = df['BsmtFinType1']*df['BsmtFinSF1']
    df['Bsmt2Cal'] = df['BsmtFinType2']*df['BsmtFinSF2']
    # por ultimo, vamos a sacar un promedio de calificacion del sotano, dividiendolo entre el total de tamaño del sotano
    df['BsmtCalTot'] = ((df['Bsmt1Cal']+df['Bsmt2Cal']+df['BsmtQual']+df['BsmtCond']+df['BsmtExposure'])/5)/(df['TotalBsmtSF'])

    # Garage:
    # Ahora Trabajamos con la de Garage
    df['GaragCal'] = ((df['GarageFinish']+df['GarageQual']+df['GarageCond'])/3)/df['GarageArea']

    # Cocina:
    # Creamos la calificacion de la cocina
    df['KitchCal'] = df['KitchenAbvGr']*df['KitchenQual']

    # Creamos la calificacion de Chimenea
    df['FireplCal'] = df['Fireplaces']+df['FireplaceQu']

    # Creamos la calificacion de la Alberca
    df['PoolCal'] = (df['PoolArea']*df['PoolQC'])/2
    
    # Nos aseguramos de no tener valores nulos
    df['BsmtCalTot'] = df['BsmtCalTot'].replace(np.nan,0)
    df['GaragCal']=df['GaragCal'].replace(np.nan,0)
    
    # Pendiente eliminar estos datos
    #['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinSF1','BsmtFinType2','BsmtFinSF2','TotalBsmtSF']
    #Columnas a eliminar restantes de todos los procesos anteriores.
    df = df.drop(['Id','KitchenAbvGr','PoolArea','PoolQC','KitchenQual','Fireplaces','FireplaceQu','GarageFinish','GarageQual','GarageCond','GarageArea','MSSubClass','MoSold','YrSold','YearRemodAdd','YearBuilt','GarageYrBlt','BsmtFullBath','FullBath','BsmtHalfBath','HalfBath','TotRmsAbvGrd','BsmtUnfSF','LowQualFinSF','GrLivArea'],axis=1)
    
    return df

def regresion_lineal(df):
    # generamos las variables
    
    # Pendiente separar los datos
    y = df['SalePrice']
    x1 = df.drop('SalePrice',axis=1)

    # Agregamos la constante
    X = sm.add_constant(x1)

    # Generamos el modelo
    linreg_stats = sm.OLS(y, X).fit()

    return linreg_stats.summary()

In [48]:
df = pd.read_csv('train.csv')


In [49]:
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [50]:
df = estandarizacion_informacion(df)

In [51]:
df

,LotFrontage,LotArea,LotShape,Utilities,LandSlope,OverallQual,OverallCond,MasVnrArea,ExterQual,ExterCond,...,SaleCondition_Partial,TotBaños,TotMedBaño,Bsmt1Cal,Bsmt2Cal,BsmtCalTot,GaragCal,KitchCal,FireplCal,PoolCal
0,65.0,8450,3,4,2,7,5,196.0,3,2,...,0,3,1,4236,0,0.991121,0.003650,3,0,0.0
1,80.0,9600,3,4,2,6,8,0.0,2,2,...,0,2,1,4890,0,0.776228,0.004348,2,3,0.0
2,68.0,11250,2,4,2,7,5,162.0,3,2,...,0,3,1,2916,0,0.635435,0.003289,3,3,0.0
3,60.0,9550,2,4,2,7,5,0.0,2,2,...,0,2,0,1080,0,0.287302,0.002596,3,4,0.0
4,84.0,14260,2,4,2,8,5,350.0,3,2,...,0,3,1,3930,0,0.687860,0.002392,3,3,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,62.0,7917,3,4,2,6,5,0.0,2,2,...,0,2,1,0,0,0.001259,0.004348,2,3,0.0
1456,85.0,13175,3,4,2,6,6,119.0,2,2,...,0,3,0,3950,489,0.576524,0.003333,2,4,0.0
1457,66.0,9042,3,4,2,7,9,0.0,4,3,...,0,2,0,1650,0,0.287500,0.007937,3,5,0.0
1458,68.0,9717,3,4,2,5,6,0.0,2,2,...,0,2,0,294,3087,0.628386,0.006944,3,0,0.0


In [52]:
regresion_lineal(df)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.918
Model:                            OLS   Adj. R-squared:                  0.906
Method:                 Least Squares   F-statistic:                     74.26
Date:                Fri, 22 Jan 2021   Prob (F-statistic):               0.00
Time:                        14:36:39   Log-Likelihood:                -16707.
No. Observations:                1459   AIC:                         3.380e+04
Df Residuals:                    1267   BIC:                         3.481e+04
Df Model:                         191                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -5.455e+04    1.1e+04     -4.968      0.000   -7.61e+04    -3.3e+04
LotFrontage              16.1453     23.751      0.680      0.497     -30.450      62.740
LotArea                   0.5568      0.095      5.882      0.000       0.371       0.742
LotShape               -624.8191   1452.367     -0.430      0.667   -3474.129    2224.491
Utilities              1.882e+04   1.36e+04      1.381      0.168   -7918.944    4.56e+04
LandSlope             -1100.1575   3640.458     -0.302      0.763   -8242.147    6041.833
OverallQual            8176.4507   1056.764      7.737      0.000    6103.252    1.02e+04
OverallCond            5388.6833    848.331      6.352      0.000    3724.394    7052.972
MasVnrArea               32.1318      6.023      5.335      0.000      20.315      43.949
ExterQual              5416.2177   2190.999      2.472      0.014    1117.832    9714.603
ExterCond             -3247.8069   2215.469     -1.466      0.143   -7594.198    1098.585
BsmtQual               3289.6830   1855.879      1.773      0.077    -351.252    6930.617
BsmtCond              -3171.1308   2425.450     -1.307      0.191   -7929.472    1587.210
BsmtExposure           4468.2376   1042.249      4.287      0.000    2423.513    6512.962
BsmtFinType1           -485.0199    666.935     -0.727      0.467   -1793.438     823.398
BsmtFinSF1              -13.7533      7.334     -1.875      0.061     -28.142       0.635
BsmtFinType2           -730.2197   1497.947     -0.487      0.626   -3668.949    2208.509
BsmtFinSF2              -27.6227     13.272     -2.081      0.038     -53.659      -1.586
TotalBsmtSF              11.7778      4.774      2.467      0.014       2.413      21.143
HeatingQC              1063.9866   1010.363      1.053      0.293    -918.183    3046.156
1stFlrSF                 55.2120      4.896     11.277      0.000      45.607      64.817
2ndFlrSF                 68.1005      5.265     12.935      0.000      57.772      78.429
BedroomAbvGr          -3464.1436   1283.361     -2.699      0.007   -5981.891    -946.396
Functional             3924.6597    859.874      4.564      0.000    2237.727    5611.593
GarageCars             1.116e+04   2191.888      5.093      0.000    6862.705    1.55e+04
WoodDeckSF                9.7573      6.166      1.582      0.114      -2.340      21.854
OpenPorchSF              15.7719     11.945      1.320      0.187      -7.662      39.206
EnclosedPorch             3.8838     12.998      0.299      0.765     -21.617      29.385
3SsnPorch                33.4712     23.701      1.412      0.158     -13.027      79.969
ScreenPorch              48.7730     13.029      3.743      0.000      23.212      74.334
Fence                  -321.4562    637.426     -0.504      0.614   -1571.984     929.071
MiscVal                